## Hyperparameter search on NACA0012 data
Do this tutorial after [`run.ipynb`](./run.ipynb).

You might be wondering how we arrived at the input configurations in the 'Optimizing' and the 'Improving run time' sections in `run.ipynb`. The method `deepk.hyp_search:run_hyp_search()` can perform hyperparameter search on either `StatePredictor` or `TrajectoryPredictor`. This sweeps the values of the different inputs to the class and its methods. Each configuration is trained and the loss and ANAE statistics across epochs recorded for training data (and validation data, if provided). These results can then be used to select 'good' input configurations.

In [1]:
from deepk.hyp_search import run_hyp_search
from deepk.state_predictor import StatePredictor_DataHandler
from deepk import utils

### Load data

In [2]:
import pickle
with open('./data.pkl', 'rb') as f:
    data = pickle.load(f)

The type of Data Handler we create determines the predictor model on which hyperparameter search is performed. In this case, we create a `StatePredictor_DataHandler`. This automatically sets the hyperparameter search to run on `StatePredictor` models.

In [3]:
dh = StatePredictor_DataHandler(
    Xtr=data['Xtr'], ttr=data['ttr'],
    Xva=data['Xva'], tva=data['tva']
)

**The data handler is the only part that has to change when switching between `TrajectoryPredictor` and `StatePredictor`. Everything else in the hyperparameter search remains the same.**

Note that test data is not used in hyperparameter search. It is highly recommended to provide validaion data, so that validation loss and ANAE statistics can also be collected.

The following is an optional step used to seed the run. Since neural nets initialize their parameters randomly, setting the same random seed will ensure that your results are exactly the same as this tutorial.

In [15]:
utils.set_seed(10)

### Define options for hyperparameter search
The `StatePredictor` class has possible inputs:
- `dh`
- `rank`
- `encoded_size`
- `encoder_hidden_layers`
- `decoder_hidden_layers`
- `batch_norm`

Its `train_net()` method has possible inputs:
- `numepochs`
- `early_stopping`
- `early_stopping_metric`
- `lr`
- `weight_decay`
- `decoder_loss_weight`
- `Kreg`
- `cond_threshold`
- `clip_grad_norm`
- `clip_grad_value`

All of these inputs, with the exception of `dh` already defined above, can be swept together in the hyperparameter search.

For more on the inputs, see the [documentation of `StatePredictor`](htps://TODO).

Let us define some values to sweep over.

In [16]:
hyp_options = {
    'rank': 6, # 1 option
    'encoder_hidden_layers': [ [200,200], [300,200,100] ], # 2 options
    'numepochs': [200, 300, 400], # 3 options
    'clip_grad_value': 2. # 1 option
}

Options that are not provided will revert to defaults.

### Run hyperparameter search
Let us now run the hyperparameter search.

In [17]:
try:
    run_hyp_search(
        dh = dh,
        hyp_options = hyp_options
    )
except ValueError as e:
    print(f'ValueError: {e}')

ValueError: 'encoded_size' is a required argument to StatePredictor, so 'hyp_options' must include a key-value pair for {'encoded_size': <encoded_size_values>}


Uh oh, you got a `ValueError: 'encoded_size' is a required argument to StatePredictor, so 'hyp_options' must include a key-value pair for {'encoded_size': <encoded_size_values>}`. As it says, we need to specify one or more options for `encoded_size` since this is a required argument that does not have a default value. Let us specify:

In [18]:
hyp_options['encoded_size'] = [50, 100]

`hyp_options` now has a total of 12 options = 2 for `'encoded_size'` times 2 for `'encoder_hidden_layers'` times 3 for `'numepochs'`.

Let us run it again. This may take a minute.

In [19]:
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options
)

  0%|          | 0/12 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_55duKHWjD3tBEJ8KYFKUh8/hyp_search_results.csv.

Performing total 12 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperparameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 6
encoded_size : 50, 100
encoder_hidden_layers : [200, 200], [300, 200, 100]
numepochs : 200, 300, 400
clip_grad_value : 2.0
********************************************************************************



100%|██████████| 12/12 [00:55<00:00,  4.59s/it]


### Results
Hooray, it now runs successfully and returns a path to a folder. This contains a file `hyp_search_results.csv`. Let us open it.

In [20]:
import pandas as pd
import os
df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df

,UUID,encoded_size,encoder_hidden_layers,numepochs,avg_recon_loss_tr,final_recon_loss_tr,avg_recon_loss_va,best_recon_loss_va,bestep_recon_loss_va,avg_lin_loss_tr,...,avg_lin_anae_tr,final_lin_anae_tr,avg_lin_anae_va,best_lin_anae_va,bestep_lin_anae_va,avg_pred_anae_tr,final_pred_anae_tr,avg_pred_anae_va,best_pred_anae_va,bestep_pred_anae_va
0,iNnxrPZqsiVfattShWSJnt,100,"[300, 200, 100]",400,0.000662,6.831592e-07,0.003407,0.000135,322,0.000077,...,3.213051,1.307821,7.096824,1.033443,87,49.102683,8.744789,78.046216,18.098024,356
1,VNQ599h7fmwLxEabNiur4j,50,"[300, 200, 100]",400,0.000685,1.190653e-06,0.003589,0.000160,348,0.000087,...,4.956933,19.861971,10.353928,1.190545,74,55.833751,7.745581,103.012294,23.051123,391
2,9DdHdUXNvTZ42eAD9JKDJM,100,"[200, 200]",400,0.000781,9.123873e-07,0.003476,0.000135,380,0.000036,...,3.210110,1.320593,12.178429,2.085934,121,65.876381,11.496212,113.172444,26.812935,368
3,h5Duqb498YYPmMu52ZFFgx,50,"[200, 200]",400,0.000932,5.487989e-06,0.004780,0.000445,371,0.000057,...,9.266469,5.436409,35.501688,2.204152,78,72.703215,19.305519,114.892749,31.420439,396
4,SHbzxwzFYJ9HMUNxGnjsGH,100,"[300, 200, 100]",300,0.000990,1.065168e-06,0.005076,0.000099,287,0.000160,...,4.594642,0.739944,29.717091,1.661546,207,61.990227,7.649416,123.586224,26.376110,292
5,Qh8KpbR9TdBrV8RkdyH3q2,50,"[300, 200, 100]",300,0.001008,3.126138e-06,0.005544,0.000278,274,0.000052,...,3.245654,0.788686,17.093329,1.379037,137,65.601783,9.741706,125.932823,30.363571,288
6,FjP9gswbejuxhv39ey5zXv,100,"[200, 200]",300,0.001125,1.288227e-05,0.005652,0.000382,299,0.000081,...,8.077429,6.767651,42.054641,2.216914,85,96.714408,23.684887,143.633718,33.752071,206
7,5yQdBb3mw9ZTSmVWcHcU5Q,50,"[300, 200, 100]",200,0.001393,2.282682e-06,0.007165,0.000248,154,0.000245,...,7.040722,1.878384,21.883381,1.364759,67,92.357070,12.237989,146.667590,33.520821,200
8,9QzfMJoFUzT6mKNo7o6Dcj,100,"[300, 200, 100]",200,0.001396,1.678960e-06,0.006861,0.000217,199,0.000174,...,5.635093,0.556354,12.895879,1.357195,99,87.150974,11.498751,149.739165,22.095259,189
9,8bW9GKnReq2bTiJRAP79yt,100,"[200, 200]",200,0.001553,5.191852e-06,0.007210,0.000278,186,0.000039,...,4.372336,0.951696,15.026154,2.647733,172,114.426364,20.360687,149.997272,34.385357,199


This contains loss and ANAE statistics for all 12 runs, as `encoded_size`, `encoder_hidden_layers`, and `numepochs` are swept. The statistics collected for each performance metric `<perf>` are:
- `avg_<perf>_tr` - Average of metric for training data over all epochs.
- `final_<perf>_tr` - Value of metric in last epoch of training data.

If validation data is provided:
- `avg_<perf>_va` - Average of metric for validation data over all epochs.
- `best_<perf>_va` - Best value of metric for validation data over all epochs.
- `bestep_<perf>_va` - Epoch at which best value of metric for validation data was obtained.

The 12 results from top to bottom are arranged from best to worst of the `sort_key` of `run_hyp_search()`, which is by default set to `'avg_pred_anae_va'`. This is because prediction ANAE of validation data averaged across all epochs is an important metric for quantifying performance.

Let's view this.

In [21]:
df_truncated = df[['encoded_size', 'encoder_hidden_layers', 'numepochs', 'avg_pred_anae_va']]
df_truncated

,encoded_size,encoder_hidden_layers,numepochs,avg_pred_anae_va
0,100,"[300, 200, 100]",400,78.046216
1,50,"[300, 200, 100]",400,103.012294
2,100,"[200, 200]",400,113.172444
3,50,"[200, 200]",400,114.892749
4,100,"[300, 200, 100]",300,123.586224
5,50,"[300, 200, 100]",300,125.932823
6,100,"[200, 200]",300,143.633718
7,50,"[300, 200, 100]",200,146.667590
8,100,"[300, 200, 100]",200,149.739165
9,100,"[200, 200]",200,149.997272


### Ignoring first few epochs
The ANAE values above are really high. This is because the performance is erratic early on, before settling down. You can also see this by looking back at the results in [`run.ipynb`](./run.ipynb):

<img src="./skewed_initial_epochs_example.png" width="300"/>

A few erratic initial epochs can skew the average statistics significantly. This is why the `run_hyp_search()` method has an argument `avg_ignore_initial_epochs`, which specifies the number of initial epochs to ignore for average calculations. Let us set this to `100`, so that averaging starts from the 100th epoch.

In [22]:
utils.set_seed(10)

output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options,
    avg_ignore_initial_epochs = 100
)

df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df_truncated = df[['encoded_size', 'encoder_hidden_layers', 'numepochs', 'avg_pred_anae_va']]
df_truncated

  0%|          | 0/12 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_9F87fXxvyyckxba7qX57Tq/hyp_search_results.csv.

Performing total 12 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperparameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 6
encoded_size : 50, 100
encoder_hidden_layers : [200, 200], [300, 200, 100]
numepochs : 200, 300, 400
clip_grad_value : 2.0
********************************************************************************



100%|██████████| 12/12 [00:54<00:00,  4.51s/it]


,encoded_size,encoder_hidden_layers,numepochs,avg_pred_anae_va
0,100,"[300, 200, 100]",400,30.929258
1,50,"[300, 200, 100]",400,37.337053
2,100,"[300, 200, 100]",200,37.530303
3,100,"[300, 200, 100]",300,43.026576
4,100,"[200, 200]",400,48.322604
5,50,"[300, 200, 100]",200,48.949412
6,100,"[200, 200]",200,54.143983
7,50,"[300, 200, 100]",300,55.724311
8,100,"[200, 200]",300,64.380776
9,50,"[200, 200]",400,67.268883


The results look a lot better now, since they are less sensitive to outlier epochs. You can see from these results that `encoder_hidden_layers = [300, 200, 100]` is doing better than `[200, 200]`. These insights are very helpful in selecting a good combination of hyperparameters.

### Controlling the number of runs
If you don't want to wait to run every possible configuration (which can exponentially explode as the number of options increase), you can control the number of runs using the `numruns` argument of `run_hyp_search()`. Let us set this to 5. This will randomly sample 5 runs out of the total of 12.

In [23]:
utils.set_seed(10)

output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options,
    avg_ignore_initial_epochs = 100,
    numruns = 5
)

df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df_truncated = df[['encoded_size', 'encoder_hidden_layers', 'numepochs', 'avg_pred_anae_va']]
df_truncated

  0%|          | 0/5 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_AQLMLHcZKUYGM6c5qoDMQS/hyp_search_results.csv.

Performing total 5 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperparameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 6
encoded_size : 50, 100
encoder_hidden_layers : [200, 200], [300, 200, 100]
numepochs : 200, 300, 400
clip_grad_value : 2.0
********************************************************************************



100%|██████████| 5/5 [00:21<00:00,  4.39s/it]


,encoded_size,encoder_hidden_layers,numepochs,avg_pred_anae_va
0,50,"[300, 200, 100]",400,25.391461
1,100,"[300, 200, 100]",200,39.539752
2,100,"[200, 200]",300,41.807985
3,50,"[200, 200]",400,65.512063
4,100,"[200, 200]",200,76.329162


### Performance vs time tradeoff
We highly recommend performing hyperparameter search for any problem as it can lead to massively improved results (we got $<7\%$ prediction ANAE in [`run.ipynb`](./run.ipynb)!). The longer you perform a hyperparameter search for, the more likely you are to get good results. If required, you can perform several hundred or even several thousand runs, which can take time to run, but the results are usually worth it.

Here's an example of an extensive hyperparameter search:
```python
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = {
        'rank': [3,6,8,10,20], #5 options
        'num_encoded_states': [50,100,200,500,1000], #5 options
        'encoder_hidden_layers': [
            [100,100],[200,200],[500,500],
            [50,100],[100,50],[100,200],[200,100],[200,500],[500,200],[500,1000],[1000,500],
            [100,100,100],[200,200,200],[500,500,500],
            [50,100,200],[200,100,50],[100,200,500],[500,200,100],[200,500,1000],[1000,500,200]
        ], #20 options
        'weight_decay': [0.,1e-6,1e-5,1e-4], #4 options
        'Kreg': [0.,1e-3,1e-2], #3 options
        'clip_grad_norm': [None,5.,10.], #3 options
        'clip_grad_value': [None,2.], #2 options
    }, # total = 36,000 options
    avg_ignore_initial_epochs = 100,
    numruns = 3600 # randomly sample 10% of the entire space
)
```

### A note on `decoder_loss_weight`
Since `decoder_loss_weight` is an input to `train_net()`, it is a valid key for `hyp_options`. However, note that changing `decoder_loss_weight` will change the scale of the loss function, so it won't be fair any more to compare the loss matrics across configurations with different values of `decoder_loss_weight`. The ANAE metrics can still be compared across all configurations.